In [15]:
import requests
import pandas as pd
# Please contact scanR team to get your own API access
scanr_api_url = 'https://scanr.dataesr.ovh/api/scanr-publications/_search' 

# Which softwares mentioned for a lab ?

In [36]:
aggregations = {'bySoft': {'terms': {'field': 'softwares.id_name.keyword', 'size': 10}}}
body = {
    'query': {
        'bool': {
            'filter': [
                {
                    'term': {
                        'bso_local_affiliations.keyword': '482775' # struct id HAL
                    }
                }
            ]
         },
 },
 'from': '0',
 'size': '0',
 'aggs': aggregations
 #'_source': ['id', 'orcid', 'id_hal', 'fullName']
}
res = requests.post(scanr_api_url, json=body, timeout=600).json()

data = res['aggregations']['bySoft']['buckets']
for e in data:
    v_split = e['key'].split('###')
    e['software_id'] = v_split[0]
    e['software_name'] = v_split[1]
    e['nb_publications'] = e['doc_count']
    del e['key']
    del e['doc_count']
pd.DataFrame(data)

,software_id,software_name,nb_publications
0,Code,Code,4
1,Udpipe,Udpipe,4
2,scripts,scripts,4
3,Faiss,Faiss,3
4,Grobid,Grobid,3
5,Librispeech,Librispeech,3
6,Mbert,Mbert,3
7,Q187432,scripts,3
8,Aramorph,Aramorph,2
9,Camembert,Camembert,2


# Which labs use a software ?

In [74]:
software_name = 'coq'

aggregations = {
    'byLocalAff': {'terms': {'field': 'bso_local_affiliations.keyword', 'size': 10}},
    'byAff': {'terms': {'field': 'affiliations.id_name.keyword', 'size': 1000}},
}
body = {
    'query': {
        'bool': {
            'filter': [
                {
                    'term': {
                        'softwares.softwareName': software_name
                    }
                }
            ]
         },
 },
 'from': '0',
 'size': '0',
 'aggs': aggregations
 #'_source': ['id', 'orcid', 'id_hal', 'fullName']
}
res = requests.post(scanr_api_url, json=body, timeout=600).json()

data = res['aggregations']['byLocalAff']['buckets']
for e in data:
    e['nb_publications'] = e['doc_count']
    del e['doc_count']
df_local_aff = pd.DataFrame(data)

data = res['aggregations']['byAff']['buckets']
new_data = []
for e in data:
    v_split = e['key'].split('###')
    e['affiliation_id'] = v_split[0]
    e['affiliation_name'] = v_split[1].split('|||')[0].split('_')[1]
    e['nb_publications'] = e['doc_count']
    del e['key']
    del e['doc_count']
    if len(e['affiliation_id']) == 10: # filter on labs only
        new_data.append(e)
df_aff = pd.DataFrame(new_data)

In [76]:
df_aff.head(25)

,affiliation_id,affiliation_name,nb_publications
0,198018249C,Centre Inria de l’Université de Rennes,30
1,200012163A,Institut de Recherche en Informatique et Systè...,28
2,196718247G,Centre Inria de Paris,24
3,199812948M,Laboratoire de recherche en informatique,19
4,200818248E,Centre de recherche Inria Saclay - Île-de-France,19
5,198318250R,Centre Inria d'Université Côte d'Azur,17
6,200918993K,Certification de logiciel par analyse sémantique,15
7,201221053L,"Certified programs, certified tools, certified...",15
8,199511789R,Verimag,14
9,199511949P,Institut de recherche en informatique de Toulouse,13
